In [4]:
import camembert
import torch
from setuptools import setup, Extension
import torch.utils


In [11]:
# Load the model in fairseq
from fairseq.models.roberta import CamembertModel
camembert = CamembertModel.from_pretrained('camembert.v0')
camembert.eval()  # disable dropout (or leave in train mode to finetune)

RobertaHubInterface(
  (model): RobertaModel(
    (decoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (embed_tokens): Embedding(32005, 768, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 768, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
            (final_layer_norm): LayerNorm((768

In [12]:
masked_line = 'Le camembert est <mask> :)'
camembert.fill_mask(masked_line, topk=5)

[('Le camembert est délicieux :)', 0.4909121096134186, ' délicieux'),
 ('Le camembert est excellent :)', 0.10556938499212265, ' excellent'),
 ('Le camembert est succulent :)', 0.03453318029642105, ' succulent'),
 ('Le camembert est meilleur :)', 0.03303113952279091, ' meilleur'),
 ('Le camembert est parfait :)', 0.03007645532488823, ' parfait')]

In [13]:
line = "J'aime le camembert!"
tokens = camembert.encode(line)
last_layer_features = camembert.extract_features(tokens)
assert last_layer_features.size() == torch.Size([1, 10, 768])

# Extract all layer's features (layer 0 is the embedding layer)
all_layers = camembert.extract_features(tokens, return_all_hiddens=True)
assert len(all_layers) == 13
assert torch.all(all_layers[-1] == last_layer_features)